In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Github/dus_mm/')
import numpy as np
import pandas as pd
import utils
import glob

from matplotlib import pyplot as plt
import seaborn as sb
os.chdir('data')

columnsX = ['freq','AX','BX','BY','CY','CX','DX','DY','AY']
columnsY = ['AX','BX','BY','CY','CX','DX','DY','AY','AQ','AL','BQ','BL','CQ','CL','DQ','DL']
columnsYout = ['AQ','AL','BQ','BL','CQ','CL','DQ','DL','X','Y','Q','L']

Mounted at /content/drive


In [2]:
X = pd.DataFrame()
y = pd.DataFrame()
numb_files = len(glob.glob('data_output*'))
for i in glob.glob('data_output*'): 
    numb = i[11]
    if (numb == 'data_output.txt'):
        numb = ''
    y = pd.concat([y,pd.read_table(os.path.join('data_output%s.txt' % numb), header = 0, names = columnsY, 
                                   index_col = False, sep='\s+', engine='python', dtype ='float')], ignore_index = 1)
    X = pd.concat([X,pd.read_table(os.path.join('data_input%s.txt' % numb), header = 0, names = columnsX, 
                                   index_col = False, sep='\s+', engine='python', dtype ="float")], ignore_index = 1)
X = X.loc[:1000*4+3]
y = y.loc[:1000]
X = utils.init_in(X)
y = utils.decomposition(y)

In [ ]:
class Layer():
    def __init__(self, neurons: int = 256, activation: str = 'linear'):
        self.neurons = neurons
        self.activation = activation

In [ ]:
class NN_model():
  
    def __init__(self, input_dimension: int):
        self.sequential = [Layer(input_dimension)]
        self.theta = np.array([])
        self.loss = 'mean_squared_error'
        self.optimizer = 'grad_descent' #'adam'
        self.max_epoch = 100
        self.chunk_size = 30
        self.beta_1 = 0.9
        self.beta_2 = 0.999
        self.epsilon = 1e-8
        self.step_size = 0.001
        self.chunk_size = 10
        self.C = 1
        self.regularization = 'l2'

    def add(self, layer: Layer):
        self.sequential.append(layer)
        layer_in = self.sequential[-2]
        layer_out = self.sequential[-1]
        gamma = np.sqrt(6)/np.sqrt(layer_in.neurons + layer_out.neurons)
        init_theta = np.random.rand(layer_out.neurons,layer_in.neurons + 1) * 2 * gamma - gamma
        self.theta = np.hstack([self.theta, init_theta.ravel()])

    def activation(self, layer: Layer, x, diff: bool = False):
        if (layer.activation == 'relu'):
            if (diff == True):
                x[x >= 0] = 1
                x[x < 0] = 0
            else:
                x[x < 0] = 0
        elif (layer.activation == 'linear'):
            if (diff == True):
                x = 1
            else:
                x = x
        elif (layer.activation == 'sigmoid'):
            if (diff == True):
                x = self.activation(layer, x) * (1 - self.activation(layer, x))
            else:
                x = 1/(1 + np.exp(-x))
        return x

    def feedforward(self, x):
        a = x
        a_out = np.array(a.ravel())
        slice_theta1 = 0
        for i in range(1, len(self.sequential)):
            layer1 = self.sequential[i - 1]
            layer2 = self.sequential[i]
            slice_theta2 = slice_theta1 + (layer1.neurons + 1) * layer2.neurons
            
            theta1 = self.theta[slice_theta1:slice_theta2].reshape(layer2.neurons, layer1.neurons + 1)
            z = np.dot(np.concatenate((np.ones([a.shape[0],1]), a), axis = 1), theta1.T)
            a = self.activation(layer2, z)
            a_out = np.hstack([a.ravel(), a_out])
            slice_theta1 = slice_theta2
        return a_out

    def backprop(self, a_out, y): 
        grad_out = np.array([])
        slice_theta1 = 0
        slice_a1 = 0
        for i in range(1, len(self.sequential)):
            layer1 = self.sequential[-i]
            layer2 = self.sequential[-(i + 1)]
            slice_theta2 = slice_theta1 + layer1.neurons * (layer2.neurons + 1)
            slice_a2 = slice_a1 + layer1.neurons * y.shape[0]
            slice_a3 = slice_a2 + layer2.neurons * y.shape[0]           
            a1 = a_out[slice_a1:slice_a2].reshape((y.shape[0], layer1.neurons))
            a2 = a_out[slice_a2:slice_a3].reshape((y.shape[0], layer2.neurons))
            a2 = np.concatenate((np.ones([a2.shape[0], 1]), a2), axis = 1)
            theta2 = np.flip(self.theta)[slice_theta1:slice_theta2].reshape(layer1.neurons, layer2.neurons + 1)
            theta2 = np.flip(theta2)
            if (i == 1):
                delta = self.compute_loss(a1, y, diff = True) * self.activation(layer1, np.dot(a2, theta2.T), diff = True)
            else:
                delta = np.dot(delta, theta1[:,1:]) * self.activation(layer1, np.dot(a2, theta2.T), diff = True)  
            grad = (1/y.shape[0]) * np.dot(delta.T, a2)
            grad_out = np.hstack([grad.ravel(), grad_out])
            
            theta1 = theta2  
            slice_theta1 = slice_theta2
            slice_a1 = slice_a2
        return grad_out

    def compute_loss(self, a_end, y, diff: bool = False):
        if (self.loss == 'mean_squared_error'):
            if (diff == True):
                loss = (a_end - y)
            else:
                loss = np.sum(np.power((a_end - y), 2))/(2 * y.size)
        elif (self.loss == 'loglos'):
            if (diff == True):
                loss = (a_end - y)/((1 - a_end) * a_end)
            else:
                loss = - np.sum(y * np.log(a_end) + (1 - y) * np.log(1 - a_end))/y.shape[0]
        return loss
    
    def add_regularization(self):
      if (regularization == 'l1'):
        return 1/self.C * np.sign(self.theta)
      elif (regularization == 'l2'):
        return 1/self.C * self.theta
      elif (regulazation == None):
        return 0
        
    def optimize(self, x, y):
        x = np.array(x)
        y = np.array(y)
        for epoch in range(self.max_epoch):
            m = np.zeros(self.theta.shape)
            v = np.zeros(self.theta.shape)
            lst =  np.arange(y.shape[0])
            np.random.shuffle(lst)
            chunks = [lst[i: i + chunk_size] for i in range(0, y.shape[0], self.chunk_size)]
            for t, chunk in enumerate(chunks):
                xi = x[chunk]
                yi = y[chunk]
                a_out = self.feedforward(xi)
                a_end = a_out[:(self.sequential[-1].neurons * yi.shape[0])]
                loss = self.compute_loss(a_end, yi.ravel())
                grad = self.backprop(a_out, yi)    
                if (self.optimizer == 'adam'):
                    m = self.beta_1 * m + (1 - self.beta_1) * grad
                    v = self.beta_2 * v + (1 - self.beta_2) * np.power(grad, 2)
                    m_hat = m / (1 - np.power(self.beta_1, t))
                    v_hat = v / (1 - np.power(self.beta_2, t))
                    self.theta = self.theta - self.step_size * m_hat / (np.sqrt(v_hat) + self.epsilon)
                elif (self.optimizer == 'grad_descent'):
                    self.theta = self.theta - self.step_size * (grad)
            print(loss)
                    
    def __repr__(self):
        return str(len(self.sequential))


In [ ]:
dus_mm = NN_model(X.shape[1])
dus_mm.add(Layer(128, 'relu'))
dus_mm.add(Layer(128, 'relu'))
dus_mm.add(Layer(128, 'relu'))
dus_mm.add(Layer(y.shape[1], 'linear'))

In [ ]:
dus_mm.optimize(X, y)

49.920798147757644
17.121344150178444
25.573576775514606
3.277243406008887
5.325682006335081
29.147739182356304
4.030169066308115
3.5313625998632943
4.827190123546538
11.723907717489018
6.831901474194114
12.438596117969526
4.014136172984357
6.810828327020062
3.6194650404556246
1.8198715173918372
7.935155392787799
13.419449439683788
2.7756077607762326
1.2609269760104234
4.07690253770542
3.9834193988993825
1.4765144977243603
4.5623490176765
96.49137995009737
8.690408882313507
3.705031590220804
10.337608687772308
5.037447883692948
8.879362805352818
8.022555767269752
5.083338164760267
0.813216519917782
7.993612153432469
0.6126735445356933
2.896305850430807
5.942716133980514
2.2513430297234884
5.585489103568513
7.696386974165523
3.3260479555220037
4.750548505390651
8.281378643428482
0.9506020276027433
4.8336026089844735
1.931783202471318
13.736914315664565
0.7167564411986166
2.5725032616959598
1.712713552994104
1.295400284053971
12.06251222428944
2.161165254976287
3.733238715969272
2.246377

In [ ]:
y.iloc[100:101]

,AQ,AL,BQ,BL,CQ,CL,DQ,DL,X,Y,Q,L
100,9.0,0.0,0.0,7.0,0.0,8.0,4.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
dus_mm.feedforward(np.array(X.iloc[100:101]))[:12]

array([ 9.43628663, -0.27928345, -0.35269964,  6.12351201,  0.48043047,
        4.84726298,  4.826827  ,  0.01375962,  0.70647113,  1.43810813,
        2.66187581,  1.41446236])

In [ ]:
import scipy.io
mat = scipy.io.loadmat('mnist.mat')
X = mat['X']
y = mat['y']
y[y == 10] = 0
y0 = np.zeros((y.size, 10))
for i in range(y.shape[0]):
    y0[i,y[i]] = 1

In [ ]:
dus_mm = NN_model(X.shape[1])
dus_mm.add(Layer(64, 'relu'))
dus_mm.add(Layer(64, 'relu'))
dus_mm.add(Layer(y0.shape[1], 'relu'))

In [ ]:
dus_mm.optimize(X, y0)

0.047929508369332456
0.04551607684301681
0.04692694694785045
0.03076654188809388
0.027457103639072878
0.03692773925365051
0.03097002466545318
0.02987936468314012
0.028149756279192372
0.024906437380650695
0.024849136562488126
0.023087946252979625
0.025028991335726065
0.01954685661551876
0.011790309904281409
0.013954605502228625
0.01712118717720718
0.019129554374707525
0.0185385022655684
0.009494881792432552
0.012188388453124685
0.012822409967347691
0.015644545630712527
0.015011187287971985
0.020599014692261483
0.007776912461192959
0.015381573722766505
0.02038537652682268
0.014555230470382591
0.022304736986948393
0.007664617076342416
0.013657313627933133
0.011532143049891908
0.01645030711233373
0.009245012170884154
0.009734328886665169
0.007905209481229044
0.008821383033123624
0.009881029481016825
0.006624248433237797
0.009485308683840047
0.014498293005496788
0.021551440937104784
0.013668019008770516
0.013774396573223484
0.00821166042765965
0.015324493202451737
0.002650555553727043
0.001

In [ ]:
dus_mm.feedforward(X[600:601,:])[:10]

array([0.        , 0.98782404, 0.05538481, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.04617544, 0.        ])

In [ ]:
y0[600]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
 a = [1,2,3]

In [ ]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])